In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

⚠️ Google Drive not mounted — using local cache (/content)
🔧 Installing FEniCSx environment...
✨ Loading FEniCSx Jupyter magic... %%fenicsx registered


---

In [2]:
%%fenicsx -np 4

"""
Complete Example: Mesh Connectivity and DOF Numbering in DOLFINx
=================================================================

This example demonstrates all concepts from the lecture using 4 MPI processes:
1. Mesh connectivity data structures (CSR format)
2. DOF assignment and entity association
3. Local-to-global DOF mapping
4. Edge orientation for RT/Nédélec elements
5. Parallel assembly and DOF layout

Usage:
    mpirun -n 4 python this_script.py
"""

import numpy as np
from mpi4py import MPI

from dolfinx import mesh, fem
from dolfinx.fem.petsc import assemble_vector, assemble_matrix
import ufl

# Get MPI information
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def print_rank0(msg):
    """Print only from rank 0 to avoid output duplication"""
    if rank == 0:
        print(msg)

def print_all(msg):
    """Print from all ranks with rank prefix"""
    print(f"[Rank {rank}] {msg}")

# ============================================================================
# Part 1: Create a Simple Mesh
# ============================================================================
print_rank0("="*70)
print_rank0("PART 1: Creating a Partitioned Triangular Mesh")
print_rank0("="*70)

# Create a 4x4 unit square mesh with triangles (will be partitioned across 4 processes)
domain = mesh.create_unit_square(comm, 4, 4, mesh.CellType.triangle)

# Get basic mesh information
tdim = domain.topology.dim  # Topological dimension (2 for triangles)
num_cells = domain.topology.index_map(tdim).size_local
num_vertices = domain.topology.index_map(0).size_local

print_rank0(f"\nMesh Information:")
print_rank0(f"  Number of MPI processes: {size}")
print_rank0(f"  Topological dimension: {tdim}")
print_all(f"  My cells: {num_cells}, My vertices: {num_vertices}")

# ============================================================================
# Part 2: Explore Connectivity Data Structures
# ============================================================================
print_rank0("\n" + "="*70)
print_rank0("PART 2: Connectivity Data Structures (CSR Format)")
print_rank0("="*70)

# Create different connectivity relations
domain.topology.create_connectivity(2, 0)  # cell -> vertex
domain.topology.create_connectivity(0, 2)  # vertex -> cell
domain.topology.create_connectivity(1, 0)  # edge -> vertex
domain.topology.create_connectivity(2, 1)  # cell -> edge

if rank == 0:
    print("\n--- (2,0) Connectivity: Cell -> Vertices ---")
    c2v = domain.topology.connectivity(2, 0)
    print(f"Array (flattened adjacency): {c2v.array[:15]}...")  # Show first 15 elements
    print(f"Offsets: {c2v.offsets[:6]}...")  # Show first 6 offsets
    print("\nInterpretation (first 3 cells on rank 0):")
    for cell in range(min(3, num_cells)):
        vertices = c2v.array[c2v.offsets[cell]:c2v.offsets[cell+1]]
        print(f"  Cell {cell} -> Vertices {vertices}")

comm.Barrier()

if rank == 0:
    print("\n--- (0,2) Connectivity: Vertex -> Cells ---")
    v2c = domain.topology.connectivity(0, 2)
    print(f"Array: {v2c.array[:15]}...")
    print(f"Offsets: {v2c.offsets[:6]}...")
    print("\nInterpretation (first 3 vertices on rank 0):")
    for vertex in range(min(3, num_vertices)):
        cells = v2c.array[v2c.offsets[vertex]:v2c.offsets[vertex+1]]
        print(f"  Vertex {vertex} -> Cells {cells}")

comm.Barrier()

print_all(f"Number of edges (local): {domain.topology.index_map(1).size_local}")

# ============================================================================
# Part 3: P1 Lagrange DOFs (Vertex-based)
# ============================================================================
print_rank0("\n" + "="*70)
print_rank0("PART 3: P1 Lagrange Element (DOFs on Vertices)")
print_rank0("="*70)

V_P1 = fem.functionspace(domain, ("Lagrange", 1))

print_rank0(f"\nFunction Space Information:")
print_rank0(f"  Element family: Lagrange")
print_rank0(f"  Degree: 1")
print_rank0(f"  DOFs per cell: {V_P1.dofmap.dof_layout.num_dofs}")

# Show parallel DOF distribution
index_map_p1 = V_P1.dofmap.index_map
print_all(f"  Local DOFs (owned): {index_map_p1.size_local}")
print_all(f"  Ghost DOFs: {index_map_p1.num_ghosts}")
print_rank0(f"  Global DOFs (total): {index_map_p1.size_global}")

if rank == 0:
    print("\n--- Local-to-Global DOF Mapping (first 3 cells on rank 0) ---")
    print("Cell | Local DOFs -> Global DOFs")
    print("-" * 40)
    for cell in range(min(3, num_cells)):
        global_dofs = V_P1.dofmap.cell_dofs(cell)
        print(f"  {cell}  | [0,1,2] -> {global_dofs}")

# ============================================================================
# Part 4: P2 Lagrange DOFs (Vertex + Edge)
# ============================================================================
print_rank0("\n" + "="*70)
print_rank0("PART 4: P2 Lagrange Element (DOFs on Vertices and Edges)")
print_rank0("="*70)

V_P2 = fem.functionspace(domain, ("Lagrange", 2))

print_rank0(f"\nFunction Space Information:")
print_rank0(f"  Element family: Lagrange")
print_rank0(f"  Degree: 2")
print_rank0(f"  DOFs per cell: {V_P2.dofmap.dof_layout.num_dofs}")

index_map_p2 = V_P2.dofmap.index_map
print_all(f"  Local DOFs (owned): {index_map_p2.size_local}")
print_all(f"  Ghost DOFs: {index_map_p2.num_ghosts}")
print_rank0(f"  Global DOFs (total): {index_map_p2.size_global}")

if rank == 0:
    print("\n--- Local-to-Global DOF Mapping (first 2 cells on rank 0) ---")
    print("Cell | Local DOFs -> Global DOFs")
    print("-" * 50)
    for cell in range(min(2, num_cells)):
        global_dofs = V_P2.dofmap.cell_dofs(cell)
        print(f"  {cell}  | [0,1,2,3,4,5] -> {global_dofs}")

print_rank0("\n--- DOF Entity Association ---")
print_rank0("P2 triangle has:")
print_rank0("  - 1 DOF per vertex (3 total)")
print_rank0("  - 1 DOF per edge (3 total)")
print_rank0("  - Total: 6 DOFs per cell")

# ============================================================================
# Part 5: RT (Raviart-Thomas) Element - Edge DOFs with Orientation
# ============================================================================
print_rank0("\n" + "="*70)
print_rank0("PART 5: Raviart-Thomas (RT) Element (DOFs on Edges)")
print_rank0("="*70)

V_RT = fem.functionspace(domain, ("RT", 1))

print_rank0(f"\nFunction Space Information:")
print_rank0(f"  Element family: Raviart-Thomas")
print_rank0(f"  Degree: 1")
print_rank0(f"  DOFs per cell: {V_RT.dofmap.dof_layout.num_dofs}")

index_map_rt = V_RT.dofmap.index_map
print_all(f"  Local DOFs (owned): {index_map_rt.size_local}")
print_all(f"  Ghost DOFs: {index_map_rt.num_ghosts}")
print_rank0(f"  Global DOFs (total): {index_map_rt.size_global}")

if rank == 0:
    print("\n--- Local-to-Global DOF Mapping (first 2 cells on rank 0) ---")
    print("Cell | Local DOFs -> Global DOFs")
    print("-" * 40)
    for cell in range(min(2, num_cells)):
        global_dofs = V_RT.dofmap.cell_dofs(cell)
        print(f"  {cell}  | [0,1,2] -> {global_dofs}")

print_rank0("\n--- Orientation Information ---")
print_rank0("RT elements require orientation tracking for shared edges:")
print_rank0("  - Each edge has a reference orientation (canonical vertex ordering)")
print_rank0("  - Cells may see edge in opposite direction")
print_rank0("  - DOF sign is flipped accordingly during assembly")
print_rank0("  - This ensures normal flux consistency: ∫_e u·n ds")

# ============================================================================
# Part 6: Assembly Example - Mass Matrix with P1
# ============================================================================
print_rank0("\n" + "="*70)
print_rank0("PART 6: Assembly Example - Mass Matrix")
print_rank0("="*70)

# Create a simple mass matrix assembly to demonstrate parallel assembly
u = ufl.TrialFunction(V_P1)
v = ufl.TestFunction(V_P1)
a = ufl.inner(u, v) * ufl.dx

print_rank0("\nAssembling mass matrix...")
a_form = fem.form(a)

# Assemble matrix
from dolfinx.fem.petsc import assemble_matrix
A = assemble_matrix(a_form)
A.assemble()

# Get matrix information
size_local = A.getLocalSize()
size_global = A.getSize()

print_all(f"  Local matrix size: {size_local[0]} x {size_local[1]}")
print_rank0(f"  Global matrix size: {size_global[0]} x {size_global[1]}")
print_rank0(f"  Matrix assembled successfully across {size} processes")

# Show matrix non-zero structure
nnz = A.getInfo()['nz_used']
print_rank0(f"  Total non-zeros: {int(nnz)}")

print_rank0("\nThis demonstrates:")
print_rank0("  - Parallel matrix assembly using dofmap")
print_rank0("  - Each rank assembles its local cells")
print_rank0("  - PETSc handles global assembly and communication")
print_rank0("  - Ghost DOFs ensure correct off-diagonal terms")

# ============================================================================
# Part 7: Parallel DOF Layout (IndexMap)
# ============================================================================
print_rank0("\n" + "="*70)
print_rank0("PART 7: Parallel DOF Layout (IndexMap)")
print_rank0("="*70)

index_map = V_P1.dofmap.index_map

print_rank0(f"\nIndexMap Information (P1 Lagrange):")
print_all(f"  Size local (owned): {index_map.size_local}")
print_all(f"  Number of ghosts: {index_map.num_ghosts}")
print_rank0(f"  Size global: {index_map.size_global}")

print_rank0("\nParallel Distribution:")
print_rank0("  - Owned DOFs: DOFs this rank is responsible for")
print_rank0("  - Ghost DOFs: DOFs owned by neighbors, cached locally")
print_rank0("  - IndexMap handles the communication pattern")

# Show ghost information
if index_map.num_ghosts > 0:
    print_all(f"  Ghost DOF indices: {index_map.ghosts[:min(5, index_map.num_ghosts)]}...")

# ============================================================================
# Part 8: Summary Visualization
# ============================================================================
print("\n" + "="*70)
print("SUMMARY: Key Concepts Demonstrated")
print("="*70)

summary = """
1. CONNECTIVITY DATA STRUCTURES (CSR Format)
   ✓ Stored as (array, offsets) pairs
   ✓ Cache-friendly, zero-copy interop
   ✓ Examples: (2,0), (0,2), (1,0), (2,1)

2. DOF-ENTITY ASSOCIATION (from Basix)
   ✓ P1: DOFs on vertices
   ✓ P2: DOFs on vertices + edges
   ✓ RT: DOFs on edges (normal flux)

3. CONNECTIVITY → DOF SHARING
   ✓ Shared vertices → shared P1 DOFs
   ✓ Shared edges → shared RT DOFs
   ✓ Connectivity identifies which local DOFs are the same global DOF

4. LOCAL-TO-GLOBAL MAPPING (DofMap)
   ✓ Each cell: local [0,1,2,...] → global DOF indices
   ✓ Built using connectivity information
   ✓ Enables sparse matrix assembly

5. ORIENTATION (RT/Nédélec)
   ✓ Edge orientation matters for flux/circulation
   ✓ DOF sign correction per cell
   ✓ Ensures physical consistency

6. PARALLEL ASSEMBLY
   ✓ Mass matrix assembly across 4 processes
   ✓ Each rank assembles local cells
   ✓ PETSc handles global assembly
   ✓ Ghost DOFs ensure correctness

7. PARALLEL LAYOUT (IndexMap)
   ✓ Owned vs ghost DOFs
   ✓ Communication pattern for MPI
   ✓ Scalable assembly
"""

print(summary)

print_rank0("\n" + "="*70)
print_rank0("Example Complete!")
print_rank0(f"Executed on {size} MPI processes")
print_rank0("="*70)

PART 1: Creating a Partitioned Triangular Mesh

Mesh Information:
  Number of MPI processes: 4
  Topological dimension: 2
[Rank 0]   My cells: 8, My vertices: 7

PART 2: Connectivity Data Structures (CSR Format)

--- (2,0) Connectivity: Cell -> Vertices ---
Array (flattened adjacency): [7 0 8 0 8 1 0 2 1 8 1 3 2 1 4]...
Offsets: [ 0  3  6  9 12 15]...

Interpretation (first 3 cells on rank 0):
  Cell 0 -> Vertices [7 0 8]
  Cell 1 -> Vertices [0 8 1]
  Cell 2 -> Vertices [0 2 1]

--- (0,2) Connectivity: Vertex -> Cells ---
Array: [0 1 2 1 2 3 4 5 7 2 4 6 3 5 9]...
Offsets: [ 0  3  9 12 16 19]...

Interpretation (first 3 vertices on rank 0):
  Vertex 0 -> Cells [0 1 2]
  Vertex 1 -> Cells [1 2 3 4 5 7]
  Vertex 2 -> Cells [2 4 6]
[Rank 0] Number of edges (local): 15

PART 3: P1 Lagrange Element (DOFs on Vertices)

Function Space Information:
  Element family: Lagrange
  Degree: 1
  DOFs per cell: 3
[Rank 0]   Local DOFs (owned): 7
[Rank 0]   Ghost DOFs: 5
  Global DOFs (total): 25

--- 